# Segmenting and Clustering Neighborhoods in Toronto 
### Peer-graded assignment in Applied Data Science Capstone by IBM/Coursera

#### *Submitted by Jessiedee Mark B. Gingo*

## Table of contents
* [Installing Required Libraries](#installaiong)
* [Webscraping](#webscraping)
* [Data Cleaning](#cleaning)
* [Dataframe](#dataframe)


In this assignment, we will explore and cluster the neighborhoods in Toronto. The dataset is not readily available and we will be scraping it on Wikipedia, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

## Installing Required Libraries <a name="installation"></a>

For webscraping, we will be utilizing **BeautifulSoup** package. Our chosen parser is **lxml** and we will install **html5** as well to parse data from a website without any problem.

In [ ]:
!pip install BeautifulSoup4
!pip install lxml
!pip install html5

We will import required libraries. BeautifulSoup for webscraping, requests for GET request to a webpage, and pandas so we can prepare the data.

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

The following code is executed to source and html file is converted to text file. We then create the **BeautifulSoup** object, passing the source.

## Webscraping <a name="webscraping"></a>

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')


We will now define our column names. By inspecting the website, headers of the table is under **'th'** tag. The **'th'** tag is passed to the **find_all** method to create a list under **'th'** tag. Slicing the first 3 gives us **'Postcode', 'Borough'**, and **'Neighbourhood'.**

In [ ]:
columns = [] # Column values

for header in soup.find_all('th')[0:3]:
    header = header.text
    columns.append(header)

headers[-1] = headers[-1].strip() # Removing the \n in the last item
columns = headers
columns

Preparing the data. The rows list is created to contain the information. From the website, the data are located under **'tr'** tag. Passing to the **find_all** method. 

In [ ]:
# Rows values
rows = [] # Rows value container

for data in soup.find_all('tr')[1:]:
    data = data.text
    data = data.split('\n')
    rows.append(data)

rows = rows[0:-5] # Remove unwanted information
    
for i in range(len(rows)): # Clean data by removing the whitespace in the list
    rows[i] = rows[i][1:4]

rows

## Data Cleaning <a name="cleaning"></a>

The columns and rows are now established and the dataframe is will be created.

In [ ]:
df = pd.DataFrame(data=rows, columns=headers) # 
df

Items that is 'Not assigned' in  **Borough** column is dropped.

In [ ]:
df = df[df.Borough != 'Not assigned'].reset_index()
df = df.drop(['index'], axis=1) # redundant index column is removed

**Neighbourhood** with the same **Postalcode** is grouped and concatenated.

In [ ]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df

The **Neighbourhood** with 'Not assigned' value will be replaced by the value in **Borough** column.

In [ ]:
df_not_assigned = df[df['Neighbourhood'].str.contains("Not assigned")] # Extracting the dataframe whose Neighbourhood is 'Not assigned'

for i in list(df_not_assigned['Borough']): # 'Not assigned' value is replaced
    df = df.replace('Not assigned', i)

## Dataframe <a name="dataframe"></a>

First 12 items in the dataframe are shown below.

In [ ]:
df.head(12)

The shape of the dataframe is shown.

In [383]:
df.shape

(103, 3)